### 一、什么是继承
继承就是让类和类之间变为父子关系，子类可以直接访问（调用）父类的静态属性和方法。在 python 中，
新建的类可以继承一个或多个父类，父类又可称为基类或者派生类。

In [1]:
class ParentClass1:  #定义父类1
    pass


class ParentClass2:  #定义父类2
    pass


class SubClass1(ParentClass1):
    # 单继承，基类是ParentClass1，派生类是SubClass
    pass


class SubClass2(ParentClass1, ParentClass2):
    # python支持多继承，用逗号分隔开多个继承的类
    pass

(__main__.ParentClass1, __main__.ParentClass2)

In [2]:
SubClass1.__bases__  # 查看所有继承的父类

(__main__.ParentClass1,)

In [3]:
SubClass2.__bases__

(__main__.ParentClass1, __main__.ParentClass2)

### 二、继承的规则
1. 子类继承父类的成员变量和成员方法
2. 子类不继承父类的构造方法，能够继承父类的析构方法
3. 子类不能删除父类的成员，但可以重定义父类成员
4. 子类可以增加自己的成员

In [12]:

# python中子类继承父类成员变量之间的取值逻辑

class Person():

    def __init__(self, name, age, sex):
        self.name = "jasn"
        self.age = '18'
        self.sex = sex

    def talk(self):
        print("i want to speak something to yo!!")


class Chinese(Person):
    def __init__(self, name, age, sex, language):
        Person.__init__(self, name, age, sex)  # 用父类的name,age，sex 覆盖掉子类的属性
        self.age = age  # 覆盖掉了父类的 age,取值为子类实例中传入age参数
        self.language = "chinese"

    def talk(self):
        print("我说的是普通话！！")
        Person.talk(self)


obj = Chinese("nancy", '11', 'male', "普通话")

print(obj.name)  # 对应场景A
print(obj.age)  # 对应场景B
print(obj.language)  # 对应场景C
obj.talk()  # 对应场景D

# 总结：
# A:若父类中初始化了成员变量，子类调用父类构造方法未覆盖属性（self.name），则调用子类属性时取值为父类中初始化的成员变量；
# B:若父类中初始化了成员变量，若子类调用父类构造方法覆盖属性（self.age）则取值为子类实例中传入参数
# C:若父类未初始化该成员变量，则无论子类中有无进行对父类构造方法进行属性的覆盖，均取子类实例中传入的参数
# D:对于方法,如果子类有这个方法则直接调用，如果子类没有则去父类查找。父类没有则报错

jasn
11
chinese
我说的是普通话！！
i want to speak something to yo!!


### 三、继承的查找顺序
python 到底是如何实现继承的，对于你定义的每一个类， python 回计算出一个方法解析顺序（MRO）列表，
这个 MRO 列表就是一个简单的所有基类的线性顺序列表，例如
#### 深度优先
![](https://img-blog.csdnimg.cn/20200306212030359.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MjQ0NDY5Mw==,size_16,color_FFFFFF,t_70#pic_center)

```text
当类是经典类时候，多继承情况下，在要查找的属性不存在，会按照深度优先方式查找下去。
```
#### 广度优先

![](https://img-blog.csdnimg.cn/2020030621221088.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MjQ0NDY5Mw==,size_16,color_FFFFFF,t_70#pic_center)

```text
当类是新式类（python3中只有新式类），多继承情况下，再要查找属性不存在时，会按照广度优先的方式查找下去
```

In [21]:
''''
当类是新式类，多继承的情况下，在要查找的属性不存在时，会按照广度优先来查找
默认从左侧分支查找，如 ：class G(D,E,F):会先从 D 的分支查找 找到 D-C 然后从E-B -F 顺序查找
最后再查找 几个类所共同继承的顶级父类，如果顶级没有，则报错
'''


class A:
    def test(self):
        print('A中的test')


class B(A):
    def test(self):
        print('B中的test')


class C(A):
    def test(self):
        print('C中的test')


class D(C):
    def test(self):
        print('D中的test')


class E(B):
    def test(self):
        print('E中的test')


class F(A):
    def test(self):
        print('F中的test')


class G(D, E, F):
    def test(self):
        print('G中的test')


obj = G()
obj.test()

'''
第一次 E中的test
注释D，第二次：C中的test
注释C，第三次：E中的test
注释E，第四次：B中的test
注释B，第五次，F中的test
注释F，第六次，A中的test

所以新式类的继承关系（查找顺序）为:
obj-->G --> D-->C -->E -->B -->F -->A
也称广度优先

'''
print(F.__mro__)  #只有新式才有这个属性可以查看线性列表，经典类没有这个属性

#新式类继承顺序:F->D->B->E->C->A
#经典类继承顺序:F->D->B->A->E->C
#python3中统一都是新式类
#pyhon2中才分新式类与经典类 class A(object) 为经典类

G中的test
(<class '__main__.F'>, <class '__main__.A'>, <class 'object'>)


### 四、类属性的继承规则
1. 如果父类中有该属性，则子类会继承该属性
2. 如果子类没有覆写该属性值，则子类的该属性值等于父类的该属性值
3. 如果子类赋写了该属性值，则子类的该属性值等于父类的该属性值

In [24]:
class School:
    name = "学校统称"

class Qiseguang(School):
    pass

print(Qiseguang.name)

学校统称


In [25]:
class School:
    name = "学校统称"

class Qiseguang(School):
    name = "七色光"

print(Qiseguang.name)



七色光
